# Cyanobacteria Chlorophyll-a Detection with NDCI and OpenEO

This notebook demonstrates how to detect and quantify cyanobacteria chlorophyll-a concentrations in water bodies using Sentinel-2 imagery and the Normalized Difference Chlorophyll Index (NDCI) with the OpenEO API.
Additionally it creates the tiling web services (XYZ) used in the narrative [Chlorophyll-a Detection with NDCI](./ndci.md).

## Overview

In this notebook, we will:
1. Connect to an OpenEO backend service
2. Define an area of interest containing water bodies
3. Load Sentinel-2 L1C imagery for a specific date
4. Create a simple visualization service for natural color composite (RGB) with custom color formula
5. Calculate and create a visualization service for Aquatic Plants and Algae Index (APA)
6. Calculate and create a visualization service for NDCI and estimate chlorophyll-a concentration

# What is APA?

The Aquatic Plants and Algae Index (APA) is designed to highlight the presence of floating aquatic vegetation and algae in water bodies. It combines information from multiple spectral bands to enhance the visibility of these features.

## What is NDCI?

The Normalized Difference Chlorophyll Index (NDCI) is an optical water quality index designed to estimate chlorophyll-a concentrations in turbid productive waters. It uses the red and red-edge bands:

**NDCI = (B05 - B04) / (B05 + B04)**

Where:
- B05 (705 nm): Red Edge band, sensitive to chlorophyll backscattering
- B04 (665 nm): Red band, sensitive to chlorophyll absorption

The chlorophyll-a concentration is then estimated using an empirically derived model calibrated on synthetic data for cyanobacteria *Microcystis aeruginosa*:

**Chl-a (μg/L) = 826.57 × NDCI³ - 176.43 × NDCI² + 19 × NDCI + 4.071**

This method is particularly effective for detecting cyanobacteria blooms, which can pose significant water quality and public health concerns.

## Import Required Libraries

We begin by importing the necessary Python libraries for data processing and visualization.

In [39]:
import openeo
import matplotlib.pyplot as plt
from PIL import Image
from openeo.processes import array_create, if_, and_
from openeo.api.process import Parameter

## Connect to OpenEO Backend

Connect to the OpenEO backend and authenticate using OpenID Connect.

In [64]:
connection = openeo.connect(
    url="https://api.explorer.eopf.copernicus.eu/openeo"
).authenticate_oidc_authorization_code()

## Define Area of Interest

Define the spatial extent for our narrative. This example uses coordinates for a water body area. You can modify these coordinates to analyze any water body of interest.

In [65]:
# Lagoon of Venice, Italy to Trieste, Italy
spatial_extent = {"west": 12.0, "south": 44.5, "east": 14, "north": 46}
temporal_extent = ["2025-05-12", "2025-05-13"]  # May 2025
bounding_box = Parameter("bounding_box", default=spatial_extent, optional=True)
time = Parameter("time", default=temporal_extent, optional=True)
bands = Parameter("bands", default=["reflectance|b02", "reflectance|b03", "reflectance|b04", "reflectance|b05", "reflectance|b07", "reflectance|b08", "reflectance|b8a", "reflectance|b11", "reflectance|b12"], optional=True)

/home/emathot/Workspace/eopf-explorer/narratives/.venv/lib/python3.12/site-packages/openeo/api/process.py:48: UserWarning: Parameter without description: using name as description.
  warnings.warn("Parameter without description: using name as description.")


## Load Sentinel-2 Data

Load Sentinel-2 L1C (top-of-atmosphere reflectance) data. We need multiple bands for water detection and chlorophyll-a estimation:

- **B02** (Blue, 490 nm): For water detection indices
- **B03** (Green, 560 nm): For MNDWI and NDWI
- **B04** (Red, 665 nm): For NDCI and true color
- **B05** (Red Edge, 705 nm): For NDCI calculation
- **B07** (Red Edge, 783 nm): For FAI calculation
- **B08** (NIR, 842 nm): For water detection
- **B8A** (Narrow NIR, 865 nm): For FAI calculation
- **B11** (SWIR, 1610 nm): For MNDWI
- **B12** (SWIR, 2190 nm): For water detection

In order to have a fast response for the web services, we apply immediately a time dimension reduction using the `first` pixel selection reducer over the selected time period.

In [66]:
s2cube = connection.load_collection(
    "sentinel-2-l2a",
    temporal_extent=time,
    spatial_extent=bounding_box,
    bands=bands,
    properties={
        "eo:cloud_cover": lambda x: x < 20,
    },
)

s2cube = s2cube.process(
    "apply_pixel_selection",
    pixel_selection="first",
    data=s2cube,
)

## Create a natual color composite (RGB) visualization service

Define a custom color formula for natural color composite (RGB) visualization using Sentinel-2 bands.

In [ ]:
def natural_color_composite(data):
    return array_create([data[0], data[1], data[2]])

rgb_bands = Parameter("bands", default=["reflectance|b04", "reflectance|b03", "reflectance|b02"], optional=True)
color_formula = Parameter("color_formula", default="gamma rgb 1.3, sigmoidal rgb 6 0.1, saturation 1.2", optional=True)
rgb = s2cube.apply_dimension(dimension="bands", process=natural_color_composite)
rgb = rgb.linear_scale_range(
    input_min=0, input_max=0.8, output_min=0, output_max=255
)
rgb = rgb.apply(process="trunc")
rgb = rgb.process(
    "color_formula",
    data=rgb,
    formula=color_formula,
)
rgb = rgb.save_result("PNG")

# Create XYZ Service
service = connection.create_service(
    {
        "process_graph": rgb.flat_graph(),
        "parameters": [
            time.to_dict(),
            bounding_box.to_dict(),
            color_formula.to_dict(),
            rgb_bands.to_dict(),
        ]
    },
    id="rgb_venezia_lagoon_xyz_service",
    type="XYZ",
    title="EOPF Explorer - Venezia Lagoon - RGB Visualization",
    description="Sentinel-2 L2A RGB composite for natural color visualization over Venezia Lagoon",
    configuration={
        "tile_size": 256,
        "minzoom": 5,
        "maxzoom": 14,
        "extent": [
            spatial_extent["west"],
            spatial_extent["south"],
            spatial_extent["east"],
            spatial_extent["north"],
        ],
        "scope": "public",
    },
)
print(service.service_id)

Preflight process graph validation failed: [400] InvalidRequest: 1 validation error:
  {'type': 'missing', 'loc': ('body', 'id'), 'msg': 'Field required', 'input': {'process_graph': {'loadcollection1': {'process_id': 'load_collection', 'arguments': {'bands': {'from_parameter': 'bands'}, 'id': 'sentinel-2-l2a', 'properties': {'eo:cloud_cover': {'process_graph': {'lt1': {'process_id': 'lt', 'arguments': {'x': {'from_parameter': 'value'}, 'y': 20}, 'result': True}}}}, 'spatial_extent': {'from_parameter': 'bounding_box'}, 'temporal_extent': {'from_parameter': 'temporal_extent'}}}, 'applypixelselection1': {'process_id': 'apply_pixel_selection', 'arguments': {'data': {'from_node': 'loadcollection1'}, 'pixel_selection': 'first'}}, 'applydimension1': {'process_id': 'apply_dimension', 'arguments': {'data': {'from_node': 'applypixelselection1'}, 'dimension': 'bands', 'process': {'process_graph': {'arrayelement1': {'process_id': 'array_element', 'arguments': {'data': {'from_parameter': 'data'}, '

<Service service_id='25107b35-eb1b-4171-8a54-b610a02b9c5c'>


## Calculate Aquatic Plants and Algae Index (APA)

The APA detects floating vegetation and surface algal blooms using FAI (Floating Algae Index):

**FAI = B07 - [B04 + (B8A - B04) × (783 - 665) / (865 - 665)]**

High FAI values (> 0.08) indicate floating algae or vegetation.

We also apply some filtering to show APA values only on water and to remove false detections on clouds and land areas.

In [ ]:
# Aquatic Plants and Algae Index (APA) Visualization
def apa_viridis_visualization(data):
    """
    Apply viridis colormap to FAI values
    Input data array: [B02, B03, B04, B05, B07, B8A, B11, B12]
    """
    B02, B03, B04, B05, B08, B8A, B11 = (
        data[0],
        data[1],
        data[2],
        data[3],
        data[4],
        data[5],
        data[6],
    )

    # Water detection
    moisture = (B8A - B11) / (B8A + B11)
    NDWI = (B03 - B08) / (B03 + B08)
    water_bodies = (NDWI - moisture) / (NDWI + moisture)

    # indices to identify water plants and algae
    water_plants = (B05 - B04) / (B05 + B04)
    NIR2 = B04 + (B11 - B04) * ((832.8 - 664.6) / (1613.7 - 664.6))
    FAI = B08 - NIR2
    viridis_color = array_create([FAI * 8.5, water_plants * 5.5, NDWI * 1])

    # Cloud detection
    bRatio = (B03 - 0.175) / (0.39 - 0.175)
    NDGR = (B03 - B04) / (B03 + B04)

    # True color for land
    true_color_r = B04 * 3
    true_color_g = B03 * 3
    true_color_b = B02 * 3
    land_color = array_create([true_color_r, true_color_g, true_color_b])

    result = if_(
        and_(B11 > 0.1, bRatio > 1),
        land_color,
        if_(
            and_(B11 > 0.1, and_(bRatio > 0, NDGR > 0)),
            land_color,
            if_(
                NDWI < 0,
                if_(water_bodies > 0, land_color, viridis_color),
                viridis_color
            )
        )
    )
    return result

apa_bands = Parameter("bands", default=["reflectance|b02", "reflectance|b03", "reflectance|b04", "reflectance|b05", "reflectance|b08", "reflectance|b8a", "reflectance|b11"], optional=True)
# Apply APA on the data cube on the bands dimension
apa_image = s2cube.apply_dimension(dimension="bands", process=apa_viridis_visualization)
# Linear scale to 0-255 for RGB visualization
apa_image = apa_image.linear_scale_range(
    input_min=0, input_max=0.8, output_min=0, output_max=255
)
# Define PNG visualization
apa_image = apa_image.save_result("PNG")

## Create a XYZ Tile Service

In [ ]:
# Create XYZ Service
service = connection.create_service(
    {
        "process_graph": apa_image.flat_graph(),
        "parameters": [
            time.to_dict(),
            bounding_box.to_dict(),
            apa_bands.to_dict(),
        ]
    },
    id="fai_venezia_lagoon_xyz_service",
    type="XYZ",
    title="EOPF Explorer - Venezia Lagoon - FAI Visualization",
    description="Sentinel-2 L2A RGB composite for algal blooms detection using Floating Algae Index (FAI) over Venezia Lagoon",
    configuration={
        "tile_size": 256,
        "minzoom": 5,
        "maxzoom": 14,
        "extent": [
            spatial_extent["west"],
            spatial_extent["south"],
            spatial_extent["east"],
            spatial_extent["north"],
        ],
        "scope": "public",
    },
)
print(service.service_id)

Preflight process graph validation failed: [400] InvalidRequest: 1 validation error:
  {'type': 'missing', 'loc': ('body', 'id'), 'msg': 'Field required', 'input': {'process_graph': {'loadcollection1': {'process_id': 'load_collection', 'arguments': {'bands': ['reflectance|b02', 'reflectance|b03', 'reflectance|b04', 'reflectance|b05', 'reflectance|b07', 'reflectance|b08', 'reflectance|b8a', 'reflectance|b11', 'reflectance|b12'], 'id': 'sentinel-2-l2a', 'properties': {'eo:cloud_cover': {'process_graph': {'lt1': {'process_id': 'lt', 'arguments': {'x': {'from_parameter': 'value'}, 'y': 20}, 'result': True}}}}, 'spatial_extent': {'from_parameter': 'bounding_box'}, 'temporal_extent': {'from_parameter': 'temporal_extent'}}}, 'applypixelselection1': {'process_id': 'apply_pixel_selection', 'arguments': {'data': {'from_node': 'loadcollection1'}, 'pixel_selection': 'first'}}, 'applydimension1': {'process_id': 'apply_dimension', 'arguments': {'data': {'from_node': 'applypixelselection1'}, 'dimensi

<Service service_id='453c1856-968a-4862-94cc-fe03da7d427a'>


## Calculate NDCI and Chlorophyll-a Concentration

Calculate the Normalized Difference Chlorophyll Index and estimate chlorophyll-a concentration using the calibrated model.

### Create Color-Mapped Visualization

Create a custom color mapping based on chlorophyll-a concentrations following the original script's color scheme:
- **Blue tones**: Low concentrations (< 10 μg/L) - oligotrophic waters
- **Green tones**: Medium concentrations (10-50 μg/L) - mesotrophic waters
- **Yellow tones**: High concentrations (50-100 μg/L) - eutrophic waters
- **Orange to red**: Very high concentrations (> 100 μg/L) - hypereutrophic/bloom conditions

For non-water areas, true color is displayed. For floating algae (FAI > 0.08), red color is used.

In [67]:
def cyanobacteria_chl_a_visualization(data):
    """
    Apply CyanoLakes Chlorophyll-a visualization
    Input data array: [B02, B03, B04, B05, B08, B8A, B11, B12]
    """
    B02, B03, B04, B05, B08, B8A, B11, B12 = (
        data[0],
        data[1],
        data[2],
        data[3],
        data[4],
        data[5],
        data[6],
        data[7],
    )

    MNDWI_threshold = 0.42
    NDWI_threshold = 0.4
    filter_UABS = True
    # filter_SSI = False

    def water_body_identification(B04, B03, B02, B8A, B11, B12):
        """Identify water bodies using spectral indices."""

        # var ndvi=(nir-r)/(nir+r),mndwi=(g-swir1)/(g+swir1),ndwi=(g-nir)/(g+nir),ndwi_leaves=(nir-swir1)/(nir+swir1),aweish=b+2.5*g-1.5*(nir+swir1)-0.25*swir2,aweinsh=4*(g-swir1)-(0.25*nir+2.75*swir1);

        MNDWI_val = (B03 - B11) / (B03 + B11)
        NDWI_val = (B03 - B8A) / (B03 + B8A)
        NDVI_val = (B8A - B04) / (B8A + B04)
        NDWI_leaves = (B8A - B11) / (B8A + B11)
        AWEISH = B02 + 2.5 * B03 - 1.5 * (B8A + B11) - 0.25 * B12
        AWEINSH = 4 * (B03 - B11) - (0.25 * B8A + 2.75 * B11)

        # var dbsi=((swir1-g)/(swir1+g))-ndvi,wii=Math.pow(nir,2)/r,wri=(g+r)/(nir+swir1),puwi=5.83*g-6.57*r-30.32*nir+2.25,uwi=(g-1.1*r-5.2*nir+0.4)/Math.abs(g-1.1*r-5.2*nir),usi=0.25*(g/r)-0.57*(nir/g)-0.83*(b/g)+1;

        DBSI = ((B11 - B03) / (B11 + B03)) - NDVI_val
        # WII = (B8A**2) / B04
        # WRI = (B03 + B04) / (B8A + B11)
        # PUWI = 5.83 * B03 - 6.57 * B04 - 30.32 * B8A + 2.25
        # UWI = (B03 - 1.1 * B04 - 5.2 * B8A + 0.4) / absolute(B03 - 1.1 * B04 - 5.2 * B8A)
        # USI = 0.25 * (B03 / B04) - 0.57 * (B8A / B03) - 0.83 * (B02 / B03) + 1

        # if (mndwi>MNDWI_threshold||ndwi>NDWI_threshold||aweinsh>0.1879||aweish>0.1112||ndvi<-0.2||ndwi_leaves>1) {ws=1;}

        water = if_(
            MNDWI_val > MNDWI_threshold,
            1,
            if_(
                NDWI_val > NDWI_threshold,
                1,
                if_(
                    AWEINSH > 0.1879,
                    1,
                    if_(
                        AWEISH > 0.1112,
                        1,
                        if_(NDVI_val < -0.2, 1, if_(NDWI_leaves > 1, 1, 0)),
                    ),
                ),
            ),
        )

        # //filter urban areas [3] and bare soil [10]
        # if (filter_UABS && ws==1) {
        #     if ((aweinsh<=-0.03)||(dbsi>0)) {ws=0;}
        # }
        water = if_(
            and_(filter_UABS, (water == 1)),
            if_(AWEINSH <= -0.03, 0, if_(DBSI > 0, 0, water)),
            water,
        )

        return water

    # Water mask (1 = water, 0 = land)
    water = water_body_identification(B04, B03, B02, B8A, B11, B12)

    # Floating Algae Index
    NIR2 = B04 + (B11 - B04) * ((832.8 - 664.6) / (1613.7 - 664.6))
    FAI_val = B08 - NIR2
    # FAI_val = B07 - (B04 + (B8A - B04) * ((783.0 - 665.0) / (865.0 - 665.0)))

    # NDCI and Chlorophyll-a
    NDCI_val = (B05 - B04) / (B05 + B04)
    chl = 826.57 * (NDCI_val**3) - 176.43 * (NDCI_val**2) + 19 * NDCI_val + 4.071

    # True color for land
    true_color_r = B04 * 3
    true_color_g = B03 * 3
    true_color_b = B02 * 3

    # Create a spatial ones array to give scalar colors spatial dimensions
    spatial_ones = B04 * 0 + 1

    # Define color mapping based on chlorophyll-a concentration
    # Surface blooms (FAI > 0.08): red
    red_bloom = array_create(
        [
            spatial_ones * (233 / 255),
            spatial_ones * (72 / 255),
            spatial_ones * (21 / 255),
        ]
    )

    # Chlorophyll-a concentration color scale
    # < 0.5: deep blue
    color_0_5 = array_create(
        [spatial_ones * 0.0, spatial_ones * 0.0, spatial_ones * 1.0]
    )
    # 0.5-2.5: blue
    color_2_5 = array_create(
        [spatial_ones * 0.0, spatial_ones * (59 / 255), spatial_ones * 1.0]
    )
    # 2.5-5: blue-cyan
    color_5 = array_create(
        [
            spatial_ones * (15 / 255),
            spatial_ones * (113 / 255),
            spatial_ones * (141 / 255),
        ]
    )
    # 5-8: cyan-green
    color_8 = array_create(
        [
            spatial_ones * (13 / 255),
            spatial_ones * (141 / 255),
            spatial_ones * (103 / 255),
        ]
    )
    # 8-14: green
    color_14 = array_create(
        [
            spatial_ones * (42 / 255),
            spatial_ones * (226 / 255),
            spatial_ones * (28 / 255),
        ]
    )
    # 14-24: yellow-green
    color_24 = array_create(
        [spatial_ones * (134 / 255), spatial_ones * (247 / 255), spatial_ones * 0.0]
    )
    # 24-38: yellow
    color_38 = array_create(
        [spatial_ones * (208 / 255), spatial_ones * (240 / 255), spatial_ones * 0.0]
    )
    # 38-75: yellow-orange
    color_75 = array_create(
        [
            spatial_ones * (248 / 255),
            spatial_ones * (207 / 255),
            spatial_ones * (2 / 255),
        ]
    )
    # 75-150: orange
    color_150 = array_create(
        [
            spatial_ones * (240 / 255),
            spatial_ones * (159 / 255),
            spatial_ones * (8 / 255),
        ]
    )
    # 150-350: red-orange
    color_350 = array_create(
        [
            spatial_ones * (239 / 255),
            spatial_ones * (101 / 255),
            spatial_ones * (15 / 255),
        ]
    )
    # > 350: red
    color_max = array_create(
        [
            spatial_ones * (233 / 255),
            spatial_ones * (72 / 255),
            spatial_ones * (21 / 255),
        ]
    )

    # True color for non-water
    land_color = array_create([true_color_r, true_color_g, true_color_b])

    # Apply conditional color mapping
    # Non-water pixels: true color
    # Water with FAI > 0.08: red (floating algae)
    # Water with various chl-a levels: color gradient

    result = if_(
        water == 0,
        land_color,
        if_(
            FAI_val > 0.08,
            red_bloom,
            if_(
                chl < 0.5,
                color_0_5,
                if_(
                    chl < 2.5,
                    color_2_5,
                    if_(
                        chl < 5,
                        color_5,
                        if_(
                            chl < 8,
                            color_8,
                            if_(
                                chl < 14,
                                color_14,
                                if_(
                                    chl < 24,
                                    color_24,
                                    if_(
                                        chl < 38,
                                        color_38,
                                        if_(
                                            chl < 75,
                                            color_75,
                                            if_(
                                                chl < 150,
                                                color_150,
                                                if_(chl < 350, color_350, color_max),
                                            ),
                                        ),
                                    ),
                                ),
                            ),
                        ),
                    ),
                ),
            ),
        ),
    )

    return result

chl_a_bands = Parameter("bands", default=["reflectance|b02", "reflectance|b03", "reflectance|b04", "reflectance|b05", "reflectance|b08", "reflectance|b8a", "reflectance|b11", "reflectance|b12"], optional=True)
# Apply the visualization function
chl_a_image = s2cube.apply_dimension(
    dimension="bands", process=cyanobacteria_chl_a_visualization
)
# Save as TIFF
chl_a_image = chl_a_image.linear_scale_range(
    input_min=0, input_max=1, output_min=0, output_max=255
)
chl_a_image = chl_a_image.save_result("PNG")

## Create XYZ Tile Service

In [71]:
# Create XYZ Service
service = connection.create_service(
    {
        "process_graph": chl_a_image.flat_graph(),
        "parameters": [
            time.to_dict(),
            bounding_box.to_dict(),
            chl_a_bands.to_dict(),
        ]
    },
    id="chl_a_venezia_lagoon_xyz_service",
    type="XYZ",
    title="EOPF Explorer - Venezia Lagoon - Chlorophyll-a Visualization",
    description="Sentinel-2 L2A RGB composite for cyanobacteria and chlorophyll-a concentration visualization over Venezia Lagoon",
    configuration={
        "tile_size": 256,
        "minzoom": 5,
        "maxzoom": 14,
        "extent": [
            spatial_extent["west"],
            spatial_extent["south"],
            spatial_extent["east"],
            spatial_extent["north"],
        ],
        "scope": "public",
    },
)
print(service)
chla_service_id = service.service_id

Preflight process graph validation failed: [401] InvalidRequest: 401: Token expired


OpenEoApiError: [401] InvalidRequest: 401: Token expired

## Visualize the Result

Display the generated chlorophyll-a concentration from the deployed tiling service.

In [70]:
from io import BytesIO
import requests

response = requests.get(f"https://api.explorer.eopf.copernicus.eu/openeo/services/xyz/{chla_service_id}/tiles/9/273/183?time=%5B%222025-05-12%22%2C%222025-05-13%22%5D")
img = Image.open(BytesIO(response.content))
plt.imshow(img)
plt.axis('off')
plt.show()

NameError: name 'chla_service_id' is not defined

## Create True Color Comparison

Generate a natural color image for comparison with the chlorophyll-a detection.

In [ ]:
# Create natural color composite
natural_color_scaled = s2cube.apply_dimension(
    dimension="bands", process=lambda data: array_create([data[2], data[1], data[0]])
).linear_scale_range(input_min=0, input_max=1, output_min=0, output_max=255)
natural_color_scaled_trunc = natural_color_scaled.apply(process="trunc")
natural_color = natural_color_scaled_trunc.process(
    "color_formula",
    data=natural_color_scaled_trunc,
    formula="Gamma RGB 1.5 Sigmoidal RGB 6 0.3 Saturation 1",
)
natural_color.download("natural_color_ndci.png", format="PNG")

# Display both images side by side
img_natural = Image.open("natural_color_ndci.png")
img_chl = Image.open("ndci_cyanobacteria.png")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

ax1.imshow(img_natural)
ax1.set_title("Natural Color (RGB)", fontsize=14)
ax1.axis("off")

ax2.imshow(img_chl)
ax2.set_title("Chlorophyll-a Concentration\n(NDCI Method)", fontsize=14)
ax2.axis("off")

plt.tight_layout()
plt.show()

## Interpretation Guide

### Chlorophyll-a Concentration Levels:

| Color | Chl-a Range (μg/L) | Water Quality Status |
|-------|-------------------|----------------------|
| Deep Blue | < 0.5 | Ultra-oligotrophic (very clear) |
| Blue | 0.5 - 2.5 | Oligotrophic (clear) |
| Blue-Cyan | 2.5 - 8 | Mesotrophic (moderate) |
| Green | 8 - 24 | Eutrophic (nutrient-rich) |
| Yellow | 24 - 75 | Highly eutrophic |
| Orange | 75 - 150 | Hypereutrophic |
| Red | > 150 or FAI > 0.08 | Severe bloom / Floating algae |

### Applications:

- **Cyanobacteria Bloom Monitoring**: Early detection and tracking of harmful algal blooms
- **Water Quality Assessment**: Quantitative assessment of trophic status
- **Public Health Protection**: Identify areas with potential health risks
- **Lake Management**: Support decision-making for water treatment and management
- **Long-term Monitoring**: Track changes in water quality over time

### Model Specifications:

The NDCI model was calibrated using synthetic data with the following constraints:
- Non-algal particles (Cnap) < 10 μg/L
- CDOM absorption < 3 m⁻¹
- Chl-a concentrations < 500 μg/L
- Trained specifically on cyanobacteria *Microcystis aeruginosa*

**Model Performance:**
- Log Bias: 0.0023
- MAPE: 42.3%
- RMSE: 84.2 mg/m³
- Log RMSE: 0.99
- Relative RMSE: 95.8%

### Limitations:

1. Best suited for cyanobacteria-dominated blooms
2. May be less accurate in waters with very high turbidity or CDOM
3. Surface blooms may saturate the signal
4. L1C data used (top-of-atmosphere) - atmospheric effects present
5. Mixed phytoplankton assemblages may reduce accuracy

### Citation:

If using this algorithm, please cite:

Kravitz, J & Matthews M., 2020. Chlorophyll-a for cyanobacteria blooms from Sentinel-2. CyanoLakes.

### References:

- Hu, C. (2009). A novel ocean color index to detect floating algae in the global oceans. *Remote Sensing of Environment*, 113(10), 2118-2129.

- Kravitz, J., Matthews, M., Lain, L., Fawcett, S., & Bernard, S. (2021). Potential for high fidelity global mapping of common inland water quality products at high spatial and temporal resolutions based on a synthetic data and machine learning approach. *Frontiers in Environmental Science*, 19.

- Mishra, S., & Mishra, D. R. (2012). Normalized difference chlorophyll index: A novel model for remote estimation of chlorophyll-a concentration in turbid productive waters. *Remote Sensing of Environment*, 117, 394-406.

## Conclusion

This notebook demonstrates the implementation of the NDCI-based chlorophyll-a estimation for cyanobacteria detection using OpenEO and Sentinel-2 imagery. The method successfully:

- Identifies water bodies using multiple spectral indices
- Detects floating algae using FAI
- Estimates chlorophyll-a concentration using NDCI
- Produces color-mapped visualizations for easy interpretation

The approach can be extended to:
- Create time series animations of bloom development
- Generate statistical summaries by water body
- Integrate with early warning systems
- Support water quality modeling and forecasting
- Validate against in-situ measurements